In [1]:
from models.point_cloud import *

In [2]:
k = 200

In [3]:
embeddings_paths = [word2vec_embeddings_path, fasttext_embeddings_path, glove_embeddings_path]
filename_prefixes = ['word2vec', 'fasttext', 'glove']

In [4]:
for embeddings_path, filename_prefix in zip(embeddings_paths, filename_prefixes):
    for filter_word_embeddings in [False, True]:
        
        with open(f"C:\\Users\\its_d\\Desktop\\{filename_prefix}_{'filtered' if filter_word_embeddings else 'original'}.txt", 'w', encoding='utf-8') as file:
            file.write(f'')
                
        embeddings = load_embeddings(embeddings_path)
        embeddings.unit_normalize_all()
        
        if filter_word_embeddings:
            embeddings = filter_embeddings(embeddings)
        
        query_words = load_csv(words_path)
        keys = list(query_words['Word'])
        
        query_points = [embeddings[key] for key in keys]
        
        for point, key in zip(query_points, keys):
            most_similar = embeddings.similar_by_vector(vector=point, topn=(k + 1))
            most_similar_keys = [key for key, similarity in most_similar]
            
            with open(f"C:\\Users\\its_d\\Desktop\\{filename_prefix}_{'filtered' if filter_word_embeddings else 'original'}.txt", 'a', encoding='utf-8') as file:
                file.write(f'{key}\n')
            
            for neighborhood_size in range(10, 110, 10):
                words = [key for key in most_similar_keys[1: (neighborhood_size + 1)]]
                neighborhood = np.array([embeddings[key] for key in words])
                
                angles = dict()
                for i in range(len(neighborhood)):
                    for j in range(i + 1, len(neighborhood)):
                        point1 = neighborhood[i]
                        point2 = neighborhood[j]
                        
                        word1 = words[i]
                        word2 = words[j]
            
                        cosine_similarity = np.dot(point1, point2) / (np.linalg.norm(point1) * np.linalg.norm(point2))
                        angle_in_degrees = math.degrees(math.acos(cosine_similarity))
                        angles[(word1, word2)] = angle_in_degrees
    
                pair, neighborhood_thickness = max(angles.items(), key=lambda item: item[1])
                
                with open(f"C:\\Users\\its_d\\Desktop\\{filename_prefix}_{'filtered' if filter_word_embeddings else 'original'}.txt", 'a', encoding='utf-8') as file:
                    file.write(f'Neighborhood size = {neighborhood_size}: {neighborhood_thickness} {pair}\n')
            
            with open(f"C:\\Users\\its_d\\Desktop\\{filename_prefix}_{'filtered' if filter_word_embeddings else 'original'}.txt", 'a', encoding='utf-8') as file:
                file.write(f'\n')

KeyboardInterrupt: 